In [0]:
#@title Default title text
!echo "API Key: " && read apikey && wget -O - -o /dev/null "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=${apikey}&outputsize=full&datatype=csv" > data.csv

In [0]:
import pandas as pd
import numpy as np

data = pd.read_csv("data.csv")
data = data.drop(['high', 'low', 'open', 'volume'], axis=1)
data = data.iloc[:4*365] # get last 4 years

Firstly let's visualize stocks to see how they goes.

In [0]:
time = np.array(data["timestamp"], dtype=np.datetime64)
prize = np.array(data["close"])

In [29]:
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(x_axis_type="datetime")
p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "Price"
p.line(time, prize)

show(p)

Loading BokehJS ...

I will try polynomial regresion for this problem.

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

model = LinearRegression()

def split(arr, x):
    return arr[:x].reshape(-1, 1), arr[x:].reshape(-1, 1)

In [0]:
x_test, x_train = split(time, 365//2)
y_test, y_train = split(prize, 365//2)

polynomial_features= PolynomialFeatures()
x_poly = polynomial_features.fit_transform(x_train)
x_poly_test = polynomial_features.fit_transform(x_test)

model.fit(x_poly, y_train)
y_res_test = model.predict(x_poly_test)
y_res_train = model.predict(x_poly)

In [0]:
# Reshape matrixes

x_train = [x[0] for x in x_train]
y_res_train = [x[0] for x in y_res_train]
x_test = [x[0] for x in x_test]
y_res_test = [x[0] for x in y_res_test]

In [55]:
# Calculate mean absolute error
mae = sum([
    abs(x - y)
    for x, y in zip(y_test, y_res_test)
])[0]/len(y_test)

print(mae)

6.34417774207512


In [53]:
output_notebook()

p = figure(x_axis_type="datetime")
p.xaxis.axis_label = "Date"
p.yaxis.axis_label = "Price"
p.line(time, prize)

p.line(np.array(x_test, dtype=np.datetime64), y_res_test, color="red")
p.line(np.array(x_train, dtype=np.datetime64), y_res_train, color="red")

show(p)

Loading BokehJS ...